In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

CREATE FLAG PARAMATER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag= dbutils.widgets.get('incremental_flag')
print(incremental_flag)

#CREATING DIMENSIONS

fetch relative colum


In [0]:
df_source = spark.sql('''
select distinct(DEALER_ID) as dealer_id, DEALER_NAME from parquet.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`''')

dim model sink - initial and incremental

In [0]:
if spark.catalog.tableExists('car_catalog.gold.dim_dealer'):
    df_sink=spark.sql(
    ''' SELECT  dim_dealer_key,DEALER_ID,DEALER_NAME
    FROM car_catalog.gold.dim_dealer
     ''')
else:    df_sink=spark.sql(
    ''' SELECT 1 as dim_dealer_key,DEALER_ID,DEALER_NAME
    FROM parquet.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`
    WHERE 1=0 ''')


### Filtering new record vs old record

In [0]:
df_filter = df_source.join(df_sink,df_source.dealer_id ==df_sink.DEALER_ID,'left').select(df_source.dealer_id,df_source.DEALER_NAME,df_sink.dim_dealer_key)

### df_filter_old

In [0]:
df_filter_old = df_filter.filter(col('dim_dealer_key').isNotNull())
df_filter_old.display()

### df_filter_new

In [0]:
df_filter_new= df_filter.filter(col('dim_dealer_key').isNull())
df_filter_new.display()

### Create surrogate key

**fetch the max surrogate key from existing table**

In [0]:
if (incremental_flag == '0'):
   max_value=1
else:
    max_value_df= spark.sql("select max(dim_dealer_key) from cars_catalog.gold.dim_dealer")
    max_value= max_value_df.collect()[0][0]+1


In [0]:
df_filter_new = df_filter_new.withColumn('dim_dealer_key',max_value+ monotonically_increasing_id())
df_filter_new.display()

### final df filter - df_old+df_new

In [0]:
df_final= df_filter_new.union(df_filter_old)

In [0]:
df_final.display()

# scd-1 upsert


In [0]:
from delta import DeltaTable

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    delta_tbl = DeltaTable.forPath(spark,"abfss://gold@nvcardatalake.dfs.core.windows.net/dim_dealer")
    delta_tbl.alias("trg").merge(df_final.alias("src"),"trg.dim_dealer_key= src.dim_dealer_key")\
                            .whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    
else:
    df_final.write.format("delta").mode('overwrite').option('path',"abfss://gold@nvcardatalake.dfs.core.windows.net/dim_dealer").saveAsTable('cars_catalog.gold.dim_dealer')